In [6]:
from swarm import Swarm, Agent
from swarm.types import Result
from dotenv import load_dotenv
from typing import Dict, List, Tuple
import math
import os
import heapq

load_dotenv()

# Define the instructions for the Supply Agent
logistics_agent_instructions = """
You are a Logistics Agent responsible for coordinating the delivery of surplus items to demand locations.

You will be given a dictionary of locations, each with their surplus and demand data. This is already in the context variables, simply call the logistics_agent_match function.
Your task is to use the given functions to calculate the optimal routes and distances for the deliveries.
Then explain the results in a friendly and engaging manner.
"""

locations = {
    "locations": {
        # Suppliers
        "Grocery Store A": {
            "surplus": ["fruits", "dairy"],
            "surplus_mapping": {
                "fruits": ["strawberry", "apple"],
                "dairy": ["milk", "cheese"]
            },
            "data": {
                "lat": 40.712776,
                "lon": -74.005974,
                "address": "123 Main St, New York, NY 10001"
            }
        },
        "Bakery B": {
            "surplus": ["grains", "baked goods"],
            "surplus_mapping": {
                "grains": ["bread", "rice"],
                "baked goods": ["muffins", "cookies"]
            },
            "data": {
                "lat": 42.652580,
                "lon": -73.756232,
                "address": "456 Country Rd, Albany, NY 12207"
            }
        },
        "Seafood Market C": {
            "surplus": ["seafood"],
            "surplus_mapping": {
                "seafood": ["salmon", "shrimp"]
            },
            "data": {
                "lat": 42.886448,
                "lon": -78.878372,
                "address": "789 Market St, Buffalo, NY 14202"
            }
        },
        
        # Demanders
        "Soup Kitchen X": {
            "demand": ["fruits", "seafood"],
            "data": {
                "lat": 40.717054,
                "lon": -73.984472,
                "address": "321 Charity Ave, New York, NY 10002"
            }
        },
        "Homeless Shelter Y": {
            "demand": ["dairy", "grains"],
            "data": {
                "lat": 42.652580,
                "lon": -73.756232,
                "address": "654 Support St, Albany, NY 12205"
            }
        },
        "Community Kitchen Z": {
            "demand": ["baked goods", "seafood"],
            "data": {
                "lat": 42.886448,
                "lon": -78.878372,
                "address": "987 Hope Rd, Buffalo, NY 14203"
            }
        }
    }
}


In [7]:
def calculate_distance(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    """
    Calculate the Euclidean distance between two geographical points.
    
    Args:
        lat1, lon1: Latitude and Longitude of the first point.
        lat2, lon2: Latitude and Longitude of the second point.
    
    Returns:
        Euclidean distance as a float.
    """
    return math.sqrt((lat1 - lat2) ** 2 + (lon1 - lon2) ** 2)


In [8]:
def logistics_agent_match(
    context_variables
) -> Tuple[List[Tuple[str, str, str, List[str]]], 
           Dict[str, Dict[str, List[str]]], 
           Dict[str, List[str]]]:
    """
    Match suppliers to demanders based on category and proximity.

    Args:
        locations: Dictionary containing suppliers and demanders data.

    Returns:
        assignments: List of tuples (supplier, demander, category, list of items assigned).
        remaining_supplies: Dictionary of suppliers with their remaining surplus_mapping.
        remaining_demands: Dictionary of demanders with their remaining unmet categories.
    """
    suppliers = {}
    demanders = {}
    demand_centers = {}  # category: list of demanders
    
    locations = context_variables
    # Separate suppliers and demanders
    for name, info in locations["locations"].items():
        if "surplus_mapping" in info:
            suppliers[name] = {
                "surplus_mapping": {k: v.copy() for k, v in info["surplus_mapping"].items()},
                "data": info["data"].copy()
            }
        elif "demand" in info:
            # Count the number of demands per category
            demand_count = {}
            for category in info["demand"]:
                if category in demand_count:
                    demand_count[category] += 1
                else:
                    demand_count[category] = 1
                # Populate demand_centers
                if category in demand_centers:
                    demand_centers[category].append(name)
                else:
                    demand_centers[category] = [name]
            demanders[name] = {
                "demand_count": demand_count.copy(),
                "data": info["data"].copy()
            }
    
    # Initialize priority queue
    # Each entry is (distance, demander_name, supplier_name, category)
    pq = []
    
    # Populate the priority queue
    for category, demander_list in demand_centers.items():
        for demander in demander_list:
            if demander not in demanders:
                continue  # Skip if demander not in dataset
            if category not in demanders[demander]["demand_count"]:
                continue  # Skip if demander doesn't need this category
            for supplier, s_info in suppliers.items():
                if category in s_info["surplus_mapping"]:
                    distance = calculate_distance(
                        demanders[demander]["data"]["lat"],
                        demanders[demander]["data"]["lon"],
                        suppliers[supplier]["data"]["lat"],
                        suppliers[supplier]["data"]["lon"]
                    )
                    heapq.heappush(pq, (distance, demander, supplier, category))
    
    assignments = []  # List to store assignments
    
    # Process the priority queue
    while pq:
        distance, demander, supplier, category = heapq.heappop(pq)
        
        # Check if the supplier still has surplus in this category
        if category not in suppliers[supplier]["surplus_mapping"] or not suppliers[supplier]["surplus_mapping"][category]:
            continue  # No surplus left in this category
        
        # Check if the demander still needs this category
        if category not in demanders[demander]["demand_count"] or demanders[demander]["demand_count"][category] <= 0:
            continue  # Demand already fulfilled
        
        # Determine how many items can be assigned
        quantity_available = len(suppliers[supplier]["surplus_mapping"][category])
        quantity_needed = demanders[demander]["demand_count"][category]
        assigned_quantity = min(quantity_available, quantity_needed)
        
        # Assign specific items
        assigned_items = suppliers[supplier]["surplus_mapping"][category][:assigned_quantity]
        
        # Record the assignment
        assignments.append((supplier, demander, category, assigned_items))
        
        # Update supplier's surplus
        suppliers[supplier]["surplus_mapping"][category] = suppliers[supplier]["surplus_mapping"][category][assigned_quantity:]
        if not suppliers[supplier]["surplus_mapping"][category]:
            del suppliers[supplier]["surplus_mapping"][category]  # Remove category if no items left
        
        # Update demander's needs
        demanders[demander]["demand_count"][category] -= assigned_quantity
        if demanders[demander]["demand_count"][category] == 0:
            del demanders[demander]["demand_count"][category]  # Remove category if demand fulfilled
    
    # Prepare remaining supplies
    remaining_supplies = {}
    for supplier, s_info in suppliers.items():
        if s_info["surplus_mapping"]:
            remaining_supplies[supplier] = s_info["surplus_mapping"]
    
    # Prepare remaining demands
    remaining_demands = {}
    for demander, d_info in demanders.items():
        if d_info["demand_count"]:
            remaining_demands[demander] = list(d_info["demand_count"].keys())
            
    print(assignments)
    
    return assignments, remaining_supplies, remaining_demands


In [9]:
logistics_agent = Agent(
    name="Logistics Agent",
    instructions=logistics_agent_instructions,
    functions=[logistics_agent_match]
)

In [10]:
# Initialize the Swarm client
client = Swarm()
# Simulate user messages indicating surplus
messages = [
    {"role": "user", "content": "Help me coordinate the delivery of surplus items to demand locations."}
]

# Define the function call context
def run_logistics_agent():
    # Run the Supply Agent with the provided messages and additional arguments
    response = client.run(
        agent=logistics_agent,
        messages=messages,
        context_variables=locations,
    )
    return response

# Execute the Logistics Agent
response = run_logistics_agent()

# Output the response
print("Final Agent:", response.agent.name)
print("Context Variables:", response.context_variables)
print("\nMessages:")
for msg in response.messages:
    print(f"{msg['role']}: {msg['content']}")

[('Seafood Market C', 'Community Kitchen Z', 'seafood', ['salmon']), ('Bakery B', 'Homeless Shelter Y', 'grains', ['bread']), ('Grocery Store A', 'Soup Kitchen X', 'fruits', ['strawberry']), ('Grocery Store A', 'Homeless Shelter Y', 'dairy', ['milk']), ('Bakery B', 'Community Kitchen Z', 'baked goods', ['muffins']), ('Seafood Market C', 'Soup Kitchen X', 'seafood', ['shrimp'])]
Final Agent: Logistics Agent
Context Variables: {'locations': {'Grocery Store A': {'surplus': ['fruits', 'dairy'], 'surplus_mapping': {'fruits': ['strawberry', 'apple'], 'dairy': ['milk', 'cheese']}, 'data': {'lat': 40.712776, 'lon': -74.005974, 'address': '123 Main St, New York, NY 10001'}}, 'Bakery B': {'surplus': ['grains', 'baked goods'], 'surplus_mapping': {'grains': ['bread', 'rice'], 'baked goods': ['muffins', 'cookies']}, 'data': {'lat': 42.65258, 'lon': -73.756232, 'address': '456 Country Rd, Albany, NY 12207'}}, 'Seafood Market C': {'surplus': ['seafood'], 'surplus_mapping': {'seafood': ['salmon', 'shr

In [11]:
stream_response = client.run(
        agent=logistics_agent,
        messages=messages,
        context_variables=locations,
        stream=True
    )
for chunk in stream_response:

    if "content" in chunk and chunk['content']:
        print(chunk['content'], end="", flush=True)

[('Seafood Market C', 'Community Kitchen Z', 'seafood', ['salmon']), ('Bakery B', 'Homeless Shelter Y', 'grains', ['bread']), ('Grocery Store A', 'Soup Kitchen X', 'fruits', ['strawberry']), ('Grocery Store A', 'Homeless Shelter Y', 'dairy', ['milk']), ('Bakery B', 'Community Kitchen Z', 'baked goods', ['muffins']), ('Seafood Market C', 'Soup Kitchen X', 'seafood', ['shrimp'])]
I've coordinated the delivery of surplus items to the demand locations, and here are the exciting results:

### Assignments of Surplus to Demand

1. **Seafood Market C** is sending some fresh **salmon** to **Community Kitchen Z**. 🍣
2. **Bakery B** has some delicious **bread** headed to **Homeless Shelter Y**. 🍞
3. **Grocery Store A** is providing **strawberry** to brighten up the day at **Soup Kitchen X**. 🍓
4. **Grocery Store A** also has some refreshing **milk** for **Homeless Shelter Y**. 🥛
5. **Bakery B** will be sending delightful **muffins** to **Community Kitchen Z**. 🧁
6. **Seafood Market C** is also de